In [14]:
import     numpy     as    np ,     pandas     as     pd
from     matplotlib . pyplot     import     subplots 
from     sklearn . linear_model     import     \
(LinearRegression , 
LogisticRegression , 
Lasso )
from sklearn . preprocessing import StandardScaler
from     sklearn . model_selection     import     KFold
from     sklearn . pipeline     import     Pipeline
from     ISLP     import     load_data
from     ISLP . models     import     ModelSpec     as     MS
from     sklearn . model_selection     import    \
    (train_test_split,GridSearchCV)

import torch
from torch import nn
from torch.optim import RMSprop
from torch.utils.data import TensorDataset
from torchmetrics import (MeanAbsoluteError,R2Score)
from torchinfo import summary
from pytorch_lightning import Trainer
from pytorch_lightning.loggers import CSVLogger


from keras.models import Sequential
from keras.layers import Dense, Dropout
from sklearn.preprocessing import StandardScaler
from keras.utils import to_categorical
import tensorflow as tf
from tensorflow import keras

In [27]:
train = pd.read_csv("dataset/train.csv")
test = pd.read_csv("dataset/test.csv")

train_model_3 = MS(train.columns.drop(["ID","Age","Customer_Churn"]),intercept=False).fit(train)
test_model_3 = MS(test.columns.drop(["ID","Age","Customer_Churn"]),intercept=False).fit(test)
x_train = train_model_3.transform(train)
x_test = test_model_3.transform(test)
y_train = train["Customer_Churn"]
y_test = test["Customer_Churn"]

In [40]:
from keras.layers import Dense, Dropout
from keras import regularizers
from keras.callbacks import EarlyStopping

model = keras.Sequential([
    Dense(16, input_shape=(7,), activation="relu"),
    Dropout(0.2),
    Dense(12, activation="relu", kernel_regularizer=regularizers.l2(0.01)),
    Dropout(0.2),
    Dense(1, activation="sigmoid"),
])

model.compile(optimizer="adam",
              loss="binary_crossentropy",
              metrics=["accuracy"])

early_stopping = EarlyStopping(monitor='val_loss', patience=10)

history = model.fit(x_train, y_train,
                    epochs=100,
                    batch_size=32,
                    validation_split=0.2,
                    callbacks=[early_stopping])


Epoch 1/100


c:\Users\30243\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\core\dense.py:85: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.5151 - loss: 0.9462 - val_accuracy: 0.5720 - val_loss: 0.7702
Epoch 2/100
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 637us/step - accuracy: 0.5458 - loss: 0.7890 - val_accuracy: 0.5910 - val_loss: 0.7436
Epoch 3/100
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 637us/step - accuracy: 0.5444 - loss: 0.7668 - val_accuracy: 0.5950 - val_loss: 0.7238
Epoch 4/100
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 645us/step - accuracy: 0.5587 - loss: 0.7366 - val_accuracy: 0.5910 - val_loss: 0.7131
Epoch 5/100
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 617us/step - accuracy: 0.5610 - loss: 0.7258 - val_accuracy: 0.5920 - val_loss: 0.7056
Epoch 6/100
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 593us/step - accuracy: 0.5739 - loss: 0.7159 - val_accuracy: 0.5920 - val_loss: 0.6994
Epoch 7/100
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 637us/step - accuracy: 0.5756 - loss: 0.7065 - val_accuracy: 0.6000 - val_loss: 0.6932
Epoch 8/100
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 633us/step - accuracy: 0.5585 - loss: 0.7075 - val_ac

In [47]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, BatchNormalization
from keras.optimizers import Adam
from keras.callbacks import LearningRateScheduler, EarlyStopping
import numpy as np

def create_model(input_shape):
    model = Sequential([
        Dense(64, activation='relu', input_shape=(input_shape,)),
        BatchNormalization(),
        Dropout(0.3),
        Dense(64, activation='relu'),
        BatchNormalization(),
        Dropout(0.3),
        Dense(32, activation='relu'),
        Dropout(0.2),
        Dense(1, activation='sigmoid')
    ])
    return model
def step_decay(epoch):
    initial_lrate = 0.001
    drop = 0.5
    epochs_drop = 10.0
    lrate = initial_lrate * np.power(drop, np.floor((1+epoch)/epochs_drop))
    return lrate

lrate_scheduler = LearningRateScheduler(step_decay)
early_stopper = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Make sure this matches your dataset's feature count
model = create_model(7)  # Assuming your input features are of shape 7 now
model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])

# Proceed with training
history = model.fit(x_train, y_train, validation_split=0.2, epochs=100, batch_size=32,
                    callbacks=[lrate_scheduler, early_stopper])

test_loss, test_acc = model.evaluate(x_test, y_test)
print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_acc}")



Epoch 1/100


ValueError: Exception encountered when calling Sequential.call().

[1mInput 0 of layer "dense_2" is incompatible with the layer: expected axis -1 of input shape to have value 8, but received input with shape (32, 7)[0m

Arguments received by Sequential.call():
  • inputs=tf.Tensor(shape=(32, 7), dtype=int64)
  • training=True
  • mask=None

In [41]:
model.evaluate(x_test,y_test)

40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 357us/step - accuracy: 0.5630 - loss: 0.6613


[0.659480094909668, 0.5759999752044678]

In [42]:
yp = model.predict(x_test)
y_pred = []
for i in yp:
    if (i>=0.5):
        y_pred.append(1)
    else:
        y_pred.append(0)

40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 790us/step


In [44]:
from ISLP import confusion_table
confusion_table(y_test,y_pred)

Truth,0,1
Predicted,,
0,551,163
1,367,169


In [45]:
from sklearn.metrics import accuracy_score

# Assuming y_test is your true labels
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")


Accuracy: 0.576
